### This notebook compares the different approaches to iterate over pandas DataFrame

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

In [2]:
# Read the data
data = pd.read_csv("energy_consumption.csv")
data.shape

(8760, 5)

In [3]:
data.head()

,date_time,hour,minute,second,energy_kwh
0,2013-01-01 00:00:00,0,0,0,0.586
1,2013-01-01 01:00:00,1,0,0,0.580
2,2013-01-01 02:00:00,2,0,0,0.572
3,2013-01-01 03:00:00,3,0,0,0.596
4,2013-01-01 04:00:00,4,0,0,0.592


In [4]:
def calculate_cost(hour, kwh):
    """
    Calculates cost of electricity for given hour.
    """
    if 0 <= hour < 7:
        rate = 12
    elif 7 <= hour < 17:
        rate = 20
    else:
        rate = 28
        
    return(rate * kwh)

### Method 1: Simple Looping

In [5]:
%%timeit

energy_cost_list = []

for i in range(len(data)):
    
    # Fetch energy consumption and hour
    energy_used = data.iloc[i]['energy_kwh']
    hour = data.iloc[i]['hour']
    
    energy_cost = calculate_cost(energy_used, hour)
    energy_cost_list.append(energy_cost)
    
data['cost'] = energy_cost_list

2.7 s ± 89.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Method 2: Pandas iterrows()

In [6]:
%%timeit

energy_cost_list = []

for index, row in data.iterrows():

    # Fetch energy consumption and hour
    energy_used = row['energy_kwh']
    hour = row['hour']
    
    energy_cost = calculate_cost(energy_used, hour)
    energy_cost_list.append(energy_cost)
    
data['cost'] = energy_cost_list

818 ms ± 18.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Because iterrows returns a Series for each row, it does not preserve dtypes across the rows (dtypes are preserved across columns for DataFrames).  

In [7]:
df = pd.DataFrame([[1, 1.5]], columns=['int', 'float'])
row = next(df.iterrows())[1]
print(row)
print(row['int'].dtype)
print(df['int'].dtype)

int      1.0
float    1.5
Name: 0, dtype: float64
float64
int64


### Method 3: Pandas apply()

In [8]:
%%timeit

data['cost'] = data.apply(lambda x: calculate_cost(x['energy_kwh'], x['hour']), axis = 1)

164 ms ± 6.73 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Method 4: Pandas itertuples()

In [9]:
%%timeit

energy_cost_list = []

for row in data.itertuples():
    
    # Fetch energy consumption and hour
    energy_used = row.energy_kwh
    hour = row.hour
    
    energy_cost = calculate_cost(energy_used, hour)
    energy_cost_list.append(energy_cost)
    
data['cost'] = energy_cost_list

12.5 ms ± 367 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Method 5: Pandas Vectorization

In [10]:
%%timeit

data['cost'] = list(map(calculate_cost, data['energy_kwh'], data['hour']))

4.17 ms ± 67.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
calculate_cost_vectorizer = np.vectorize(calculate_cost)

In [12]:
%%timeit

data['cost'] = calculate_cost_vectorizer(data['energy_kwh'], data['hour'])

2.24 ms ± 132 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Method 6: Numpy Vectorization

In [13]:
%%timeit

data['cost'] = calculate_cost_vectorizer(data['energy_kwh'].values, data['hour'].values)

2.06 ms ± 40.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [14]:
%%timeit

data['cost'] = np.array(map(calculate_cost, data['energy_kwh'].values, data['hour'].values))

204 µs ± 8.34 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Method 7: Binning

In [15]:
%%timeit

cost_per_kwh = pd.cut(x = data.hour.values, bins = [0, 7, 17, 24], include_lowest = True, labels = [12, 20, 28]).astype(int)
data['cost'] = cost_per_kwh * data['energy_kwh'].values

1.02 ms ± 6.59 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Method 8: Binning + Numpy

In [16]:
%%timeit

cost = np.array([12, 20, 28])
hour_bins = np.digitize(data.hour.values, bins = [7, 17, 24])
data['cost'] = cost[hour_bins] * data['energy_kwh'].values

241 µs ± 3.17 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
